In [601]:
import torch
import pandas as pd
import numpy as np
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [602]:
device = "cpu"
print(f"Using {device} device")

Using cpu device


## Load Data

In [603]:
df = pd.read_csv('data.csv', sep=";")
df.head()

,Player1,Player2,Player3,Player4,Eichel Ober,Eichel Unter,Eichel Ass,Eichel 10,Eichel König,Eichel 9,...,SchellenOber,SchellenUnter,SchellenAss,Schellen10,SchellenKönig,Schellen9,Schellen8,Schellen7,Modus,Win
0,1,0,0,0,1,0,1,0,1,0,...,0,0,0,0,1,1,0,0,weiter,-1
1,0,1,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,weiter,-1
2,0,0,0,1,0,1,0,0,0,0,...,1,1,0,0,0,0,0,1,Farbwenz Schelle,1
3,0,0,0,1,1,0,0,0,0,0,...,1,1,0,0,0,0,0,1,Farbgeier Schelle,1
4,1,0,0,0,0,0,0,0,1,0,...,1,0,0,0,0,0,1,1,weiter,1


## Preprocessing

Wins are currently excluded. A Correlation between wins and the game mode is not trivial.

TODO: Research if wins are also helpful for training

In [604]:
train_data = df.drop(['Player1','Player2','Player3','Player4','Win','Modus'], axis=1).astype("float32")
train_data.head()

,Eichel Ober,Eichel Unter,Eichel Ass,Eichel 10,Eichel König,Eichel 9,Eichel 8,Eichel 7,Gras Ober,Gras Unter,...,Herz 8,Herz 7,SchellenOber,SchellenUnter,SchellenAss,Schellen10,SchellenKönig,Schellen9,Schellen8,Schellen7
0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [605]:
train_data_size = np.shape(np.array(train_data))
input_size = train_data_size[1]
print("Input Size: ", input_size)

Input Size:  32


In [606]:
output_data = df['Modus']
output_data.head()

0               weiter
1               weiter
2     Farbwenz Schelle
3    Farbgeier Schelle
4               weiter
Name: Modus, dtype: object

In [607]:
output_data_codes = output_data.astype("category").cat.codes
output_size = max(output_data_codes) + 1
y = output_data_codes.astype("long")
print("Output Size: ", output_size)

Output Size:  18


## Create Datasets

In [608]:
X_train, X_val, y_train, y_val = train_test_split(train_data, y, test_size=0.2, random_state=42)

In [609]:
class SelectGameDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return torch.tensor([self.X.iloc[idx]], device=device), torch.tensor(self.y.iloc[idx], device=device)


In [610]:
train_dataset = SelectGameDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [611]:
test_dataset = SelectGameDataset(X_val, y_val)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

## Define NN Classifier

In [612]:
class SelectGameNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, output_size)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [613]:
model = SelectGameNN().to(device)
print(model)

SelectGameNN(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=32, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=18, bias=True)
  )
)


## Train Model

In [614]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [615]:
criterion = nn.CrossEntropyLoss()

In [616]:
num_epochs = 10
print_freq = 500  

for epoch in range(num_epochs):
    running_loss = 0.0  
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()  
        optimizer.step()
        
        running_loss += loss.item()

        if (batch_idx + 1) % print_freq == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}] Batch [{batch_idx+1}/{len(train_loader)}] Loss: {running_loss / print_freq:.4f}")
            running_loss = 0.0  


Epoch [1/10] Batch [500/2120] Loss: 1.0062
Epoch [1/10] Batch [1000/2120] Loss: 0.6744
Epoch [1/10] Batch [1500/2120] Loss: 0.5928
Epoch [1/10] Batch [2000/2120] Loss: 0.5409
Epoch [2/10] Batch [500/2120] Loss: 0.5176
Epoch [2/10] Batch [1000/2120] Loss: 0.5057
Epoch [2/10] Batch [1500/2120] Loss: 0.4918
Epoch [2/10] Batch [2000/2120] Loss: 0.4719
Epoch [3/10] Batch [500/2120] Loss: 0.4661
Epoch [3/10] Batch [1000/2120] Loss: 0.4437
Epoch [3/10] Batch [1500/2120] Loss: 0.4462
Epoch [3/10] Batch [2000/2120] Loss: 0.4454
Epoch [4/10] Batch [500/2120] Loss: 0.4322
Epoch [4/10] Batch [1000/2120] Loss: 0.4289
Epoch [4/10] Batch [1500/2120] Loss: 0.4304
Epoch [4/10] Batch [2000/2120] Loss: 0.4238
Epoch [5/10] Batch [500/2120] Loss: 0.4259
Epoch [5/10] Batch [1000/2120] Loss: 0.4212
Epoch [5/10] Batch [1500/2120] Loss: 0.4271
Epoch [5/10] Batch [2000/2120] Loss: 0.4128
Epoch [6/10] Batch [500/2120] Loss: 0.4137
Epoch [6/10] Batch [1000/2120] Loss: 0.4180
Epoch [6/10] Batch [1500/2120] Loss: 0

## Evaluate Model

In [617]:
model.eval()

correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")

# TODO: Evaluate for seperate classes

model.train() 

Test Accuracy: 83.74%


SelectGameNN(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=32, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=18, bias=True)
  )
)

## Example Execution

In [618]:
# Define input
X = torch.tensor(train_data[0:1].values, device=device)
# Execute
out = model(X)
# Map output
output_code = torch.max(out, 1).indices[0].item()
category_mapping = output_data.astype("category").cat.categories
print("Example Output: ",category_mapping[output_code])

Example Output:  weiter
